In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Importo dataset di train e test

In [ ]:
train_dataset = pd.read_csv("train.csv")
test_dataset = pd.read_csv("test.csv")

In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
# non vi sono valori nulli nel dataset
train_dataset.info()

In [ ]:
# non vi sono valori nulli nel dataset
test_dataset.info()

In [ ]:
# double check on null values
train_dataset.isna().sum()
test_dataset.isna().sum()

# Preprocessing

In [ ]:
# customerId è unico per ogni riga, quindi non influenza 'Churn' per questo motivo possiamo dropparlo
train_dataset = train_dataset.drop('customerID', axis=1)
test_dataset = test_dataset.drop('customerID', axis=1)

In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
# il gender non sembra avere un effetto su churn, però prima di droppare la colonna faccio l'encoding e vedo la correlazione

sns.countplot(x=train_dataset["gender"],hue=train_dataset["Churn"],palette='mako');

# Encoding

In [ ]:
# creaiamo un dizionario che associa ad ogni colonna del dataset i valori che può assumere la colonna

def get_uniques(df, columns):
    return {column: list(df[column].unique()) for column in columns}

In [ ]:
# nel dizionario lasciamo solo le colonne che hanno valori di tipo "object", quindi escludiamo i valori numerici
# poiché in questa parte ci occupiamo dell'encoding, quindi i valori numerici sono già ok

def get_categorical_columns(df):
    return [column for column in df.columns if df.dtypes[column] == 'object']

In [ ]:
# adesso posso vedere i possibili valori che possono comparire in ogni colonna

get_uniques(train_dataset, get_categorical_columns(train_dataset))

In [ ]:
# 'Total charges' è una colonna che contiene numeri ma sono encoded come stringhe, devono essere convertiti
# train_dataset['TotalCharges'].astype(np.float), questo comando da un errore perché non tutti i valori presenti nella colonna
# sono numeri encoded come stringhe, ci sono dei valori nulli rappresentati come spazi ''
sorted(train_dataset['TotalCharges'].unique())

In [ ]:
# sostituisco le stringhe vuote con valori nulli
train_dataset['TotalCharges'] = train_dataset['TotalCharges'].replace(' ', np.NaN)

In [ ]:
# conto quanti righe presentano il valore null per capire come trattare la colonna
train_dataset.isna().sum()

In [ ]:
# i valori sono in tutto 7, è un valore piccolo rispetto al numero totale di righe, posso droppare queste righe
train_dataset.dropna(how = 'any', inplace = True)

In [ ]:
# adesso le stringhe possono essere trasformate in float
train_dataset['TotalCharges'] = train_dataset['TotalCharges'].astype(float)

In [ ]:
# ripeto tutto anche per il test_dataset
get_uniques(test_dataset, get_categorical_columns(test_dataset))

# sostituisco le stringhe vuote con valori nulli
test_dataset['TotalCharges'] = test_dataset['TotalCharges'].replace(' ', np.NaN)

# conto quanti righe presentano il valore null per capire come trattare la colonna
test_dataset.isna().sum()

# i valori sono in tutto 3, è un valore piccolo rispetto al numero totale di righe, posso droppare queste righe
test_dataset.dropna(how = 'any', inplace = True)

# adesso le stringhe possono essere trasformate in float
test_dataset['TotalCharges'] = test_dataset['TotalCharges'].astype(float)

In [ ]:
get_uniques(train_dataset, get_categorical_columns(train_dataset))

In [ ]:
get_uniques(test_dataset, get_categorical_columns(train_dataset))

In [ ]:
# nelle colonne 'MultipleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies'
# compaiono 3 possibili valori 'No, Yes, No internet service/ No phone service', non avere 'phone service o internet service' equivale
# a non avere il servizio specifico, per cui si possono accorpare

train_dataset['MultipleLines'] = train_dataset['MultipleLines'].replace('No phone service', 'No')

train_dataset[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
      'TechSupport', 'StreamingTV', 'StreamingMovies']] = train_dataset[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                                                                'TechSupport', 'StreamingTV', 'StreamingMovies']].replace('No internet service', 'No')

In [ ]:
# faccio lo stesso per il test_dataset

test_dataset['MultipleLines'] = test_dataset['MultipleLines'].replace('No phone service', 'No')

test_dataset[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
      'TechSupport', 'StreamingTV', 'StreamingMovies']] = test_dataset[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                                                                'TechSupport', 'StreamingTV', 'StreamingMovies']].replace('No internet service', 'No')

In [ ]:
# dividiamo le features in 3 tipologie: binary, ordinal e nominal. Per ognuna verrà fatto un encoding differente
# questo passaggio non è necessario per l'encoding, però mi piace questo ordine mentale e quindi ho creato le 3 liste e 
# faccio l'encoding una per volta, spero non sia un grande problema
# la maniera più semplice sarebbe stato creare un'unica lista con i nomi delle colonne e fare l'encoding utilizzando quella singola lista
 
binary_features = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                   'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                   'StreamingTV', 'StreamingMovies', 'PaperlessBilling']


ordinal_features = ['InternetService', 'Contract']

nominal_features = ['PaymentMethod']

# non è una feature ma la target column

target_column = 'Churn'

In [ ]:
# encoding per le colonne che contengono valori binari

labelEncoder_X = LabelEncoder()
for element in binary_features:
    train_dataset[element] = labelEncoder_X.fit_transform(train_dataset[element])
    
# encoding per le colonne che contengono valori ordinal

labelEncoder_X = LabelEncoder()
for element in ordinal_features:
    train_dataset[element] = labelEncoder_X.fit_transform(train_dataset[element])

# encoding per le colonne che contengono valori nominal

labelEncoder_X = LabelEncoder()
for element in nominal_features:
    train_dataset[element] = labelEncoder_X.fit_transform(train_dataset[element])

In [ ]:
# ripeto tutto anche per il test_dataset

labelEncoder_X = LabelEncoder()
for element in binary_features:
    test_dataset[element] = labelEncoder_X.fit_transform(test_dataset[element])
    
labelEncoder_X = LabelEncoder()
for element in ordinal_features:
    test_dataset[element] = labelEncoder_X.fit_transform(test_dataset[element])
    
labelEncoder_X = LabelEncoder()
for element in nominal_features:
    test_dataset[element] = labelEncoder_X.fit_transform(test_dataset[element])

In [ ]:
# eseguo l'encoding anche per la colonna target, sia per il train_dataset che per il test_dataset

train_dataset[target_column] = labelEncoder_X.fit_transform(train_dataset[target_column])

test_dataset[target_column] = labelEncoder_X.fit_transform(test_dataset[target_column])

In [ ]:
# una volta fatto l'encoding è possibile visualizzare l'heatmap con tutte le feature
# per capire se vi sono altre colonne che è possibile droppare vediamo la correlazione tra le diverse colonne

plt.figure(figsize=(18,10))
correlation = train_dataset.corr()
sns.heatmap(correlation, annot = True, linewidth = 2)

In [ ]:
# come detto precedentemente il gendere non influenza churn, let's drop gender column
train_dataset = train_dataset.drop('gender', axis=1)
test_dataset = test_dataset.drop('gender', axis=1)

# Splitting

In [ ]:
y_train = train_dataset['Churn']          
X_train = train_dataset.drop(['Churn'], axis=1)  

y_test = test_dataset['Churn']          
X_test = test_dataset.drop(['Churn'], axis=1)

# Training

In [ ]:
# per il training utilizzo la logistic regression

lr_model = LogisticRegression(random_state=0, max_iter = 10000)

lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
accuracy_lr = lr_model.score(X_test,y_test)

print(lr_model)
print('\n')
print("Accuracy: {:.3f}%".format(accuracy_lr*100))

print('\n')
cm_lr = confusion_matrix(y_test,lr_model.predict(X_test))
f, ax = plt.subplots(figsize = (5,5))
sns.heatmap(cm_lr, annot = True, linewidths = 0.5, color = "red", fmt = ".0f", ax=ax)
plt.xlabel("y_predicted")
plt.ylabel("y_true")
plt.title("Confusion Matrix of Logistic Regression")
plt.show()
print('\n\n')
print(classification_report(y_test,y_pred))
print('\n\n')